In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
np.set_printoptions(threshold=np.sys.maxsize)

/Users/omristeinberg-tatman/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [3]:
#define the data
FullData = pd.read_csv("HighLow.csv")[0:29999]

x, y = FullData.values[:, :-1], FullData.values[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3333)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
n_features = x_train.shape[1]

(20000, 2) (9999, 2) (20000,) (9999,)


In [4]:
# define the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(100, input_shape=(n_features,)))
model.add(tf.keras.layers.Dense(80))
model.add(tf.keras.layers.Dense(80))
model.add(tf.keras.layers.Dense(80))
model.add(tf.keras.layers.Dense(30))
model.add(tf.keras.layers.Dense(30))
model.add(tf.keras.layers.Dense(30))
model.add(tf.keras.layers.Dense(30))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(5))
model.add(tf.keras.layers.Dense(5))
model.add(tf.keras.layers.Dense(5))
model.add(tf.keras.layers.Dense(5))
model.add(tf.keras.layers.Dense(5))
model.add(tf.keras.layers.Dense(1))

predictions = model(np.asarray(x_train).astype('float32')).numpy()
#tf.nn.softmax(predictions).numpy()
loss_fn = tf.keras.losses.MeanAbsoluteError()#tf.keras.losses.SparseCategoricalCrossentropy()
loss_fn(y_train[:1], predictions).numpy()

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [7]:
history = model.fit(x_train, y_train, epochs=1)

625/625 [==============================] - 1s 1ms/step - loss: 6.6736e-04 - accuracy: 1.0000


In [55]:
with open('model_datamse.pkl', 'wb') as f:
    pickle.dump({'weights': model.get_weights(), 'history': history.history}, f)


In [5]:
# load the weights and history from the file and evaluate the model
with open('model_datamse.pkl', 'rb') as f:
    data = pickle.load(f)
model.set_weights(data['weights'])

In [5]:
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 1s 928us/step - loss: 9.7363e-09 - accuracy: 1.0000
Test loss: 9.699782310690352e-09
Test accuracy: 1.0


In [6]:
# evaluate the model on the test data
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)

# print the accuracy
print('Accuracy:', accuracy)

# create a chart of the model's predictions
predictions = model.predict(x_test)

313/313 [==============================] - 0s 917us/step - loss: 9.6998e-09 - accuracy: 1.0000
Accuracy: 1.0


In [6]:
y_pred = model.predict(x_test)
y_pred_classes = np.round(y_pred).flatten()

# compute class-wise accuracy and confusion matrix
class_0_correct = 0
class_0_total = 0
class_0_true_0 = 0
class_0_false_1 = 0
class_1_correct = 0
class_1_total = 0
class_1_true_1 = 0
class_1_false_0 = 0
for i in range(len(y_test)):
    true_class = y_test[i]
    pred_class = y_pred_classes[i]
    if true_class == 0:
        class_0_total += 1
        if pred_class == 0:
            class_0_correct += 1
            class_0_true_0 += 1
        else:
            class_0_false_1 += 1
    elif true_class == 1:
        class_1_total += 1
        if pred_class == 1:
            class_1_correct += 1
            class_1_true_1 += 1
        else:
            class_1_false_0 += 1

class_0_accuracy = class_0_correct / class_0_total
class_1_accuracy = class_1_correct / class_1_total

print("Class 0:")
print(f"  Total: {class_0_total}")
print(f"  Correct: {class_0_correct}")
print(f"  True 0: {class_0_true_0}")
print(f"  False 1: {class_0_false_1}")
print(f"  Accuracy: {class_0_accuracy}")
print("Class 1:")
print(f"  Total: {class_1_total}")
print(f"  Correct: {class_1_correct}")
print(f"  True 1: {class_1_true_1}")
print(f"  False 0: {class_1_false_0}")
print(f"  Accuracy: {class_1_accuracy}")

Class 0:
  Total: 4904
  Correct: 4904
  True 0: 4904
  False 1: 0
  Accuracy: 1.0
Class 1:
  Total: 5095
  Correct: 5095
  True 1: 5095
  False 0: 0
  Accuracy: 1.0
